In [1]:
# importing prompts 
from scai.modules.task.prompts import TASK_PROMPTS
from scai.modules.assistant.prompts import ASSISTANT_PROMPTS 
from scai.modules.user.prompts import USER_PROMPTS 
from scai.modules.meta_prompt.prompts import META_PROMPTS 

# import episode
from scai.modules.episode.episode import Episode


In [2]:
# params
params = {
    "n_user": 2,
    "n_assistant": 2,
    "system_k": 5,
    "chat_k": 5,
    "user_k": 5,
    "assistant_k": 5,
    "assistant_system_k": 1,
}

# prompts
prompts = {
    "task": TASK_PROMPTS["task_prompt_1"],
    "user": [USER_PROMPTS["user_prompt_1"], USER_PROMPTS["user_prompt_2"]],
    "assistant": [ASSISTANT_PROMPTS["assistant_prompt_1"], ASSISTANT_PROMPTS["assistant_prompt_1"]],
    "meta": META_PROMPTS["meta_prompt_1"],
}


from scai.custom_chat_models.crfm import crfmChatLLM

CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd"

assistant_llm =  crfmChatLLM(model_name="openai/gpt-4-0314", 
                   crfm_api_key=CRFM_API_KEY, 
                   max_tokens=50, # need to be careful with this one
                   num_completions=1,
                   request_timeout=10,
                   verbose=False,
                   temperature=0.5)

user_llm = crfmChatLLM(model_name="openai/gpt-4-0314",
                    crfm_api_key=CRFM_API_KEY,
                    max_tokens=50,
                    num_completions=1,
                    request_timeout=10,
                    verbose=False,
                    temperature=0.5)

meta_llm = crfmChatLLM(model_name="openai/gpt-4-0314",
                    crfm_api_key=CRFM_API_KEY,
                    max_tokens=200,
                    num_completions=1,
                    request_timeout=10,
                    verbose=False,
                    temperature=0.5)

# start system message
system_message_1 = "You are a helpful AI assistant."

episode = Episode.create(
    id="episode_1",
    name="episode_1",
    task_prompt=prompts["task"],
    user_prompts=prompts["user"],
    assistant_prompts=prompts["assistant"],
    meta_prompt=prompts["meta"],
    n_user=params["n_user"],
    # user_llm='userllm',
    user_llm=user_llm,
    n_assistant=params["n_assistant"],
    # assistant_llm='assistantllm',
    assistant_llm=assistant_llm,
    # meta_llm='metallm',
    meta_llm=meta_llm,
    system_k=params["system_k"],
    chat_k=params["chat_k"],
    user_k=params["user_k"],
    assistant_k=params["assistant_k"],
    assistant_system_k=params["assistant_system_k"],
    verbose=True, # set to False to run
)

episode.buffer.save_context(system={"content": system_message_1}, system_message_id="system_message_0")

episode.run()

System: You are a helpful AI assistant. Your are collaborating with a Human User to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: Respond within 50 tokens, using prior user messages as feedback for revision.
System: You're a serious Wikipedia editor committed to the facts. If an assertion is missing a source, it shouldn't be included. Your are collaborating with a Human User to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: assistant_response_0
Human: Rate your satisfaction with my respon

CustomConversationBufferWindowMemory(full_memory=CustomChatMessageHistory(messages=[SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}), AIMessage(content='assistant_response_0', additional_kwargs={}, example=False), HumanMessage(content='User_feedback0', additional_kwargs={}, example=False), AIMessage(content='assistant_response_1', additional_kwargs={}, example=False), HumanMessage(content='User_feedback1', additional_kwargs={}, example=False), SystemMessage(content='Shorter message.', additional_kwargs={})], message_ids=['system_message_0', 'conversation_0_assistant', 'conversation_0_user', 'conversation_1_assistant', 'conversation_1_user', 'system_message_0'], ratings=[], message_dict={'system_message_0': [{'message': SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}), 'rating': None}, {'message': SystemMessage(content='Shorter message.', additional_kwargs={}), 'rating': 'The system message is too long.'}], 'conversation_0_assi

In [3]:
episode.run()

System: Shorter message. Your are collaborating with a Human User to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
AI: assistant_response_0
Human: User_feedback0
Human: Respond within 50 tokens, using prior user messages as feedback for revision.
System: You're a serious Wikipedia editor committed to the facts. If an assertion is missing a source, it shouldn't be included. Your are collaborating with a Human User to write a Wikipedia article on the following topic: What principles should guide AI when handling topics that involve both human rights and local cultural or legal differences, like LGBTQ rights and women's rights? Should AI responses change based on the location or culture in which it's used?
Human: assistant_response_0
AI: User_fee

CustomConversationBufferWindowMemory(full_memory=CustomChatMessageHistory(messages=[SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}), AIMessage(content='assistant_response_0', additional_kwargs={}, example=False), HumanMessage(content='User_feedback0', additional_kwargs={}, example=False), AIMessage(content='assistant_response_1', additional_kwargs={}, example=False), HumanMessage(content='User_feedback1', additional_kwargs={}, example=False), SystemMessage(content='Shorter message.', additional_kwargs={}), AIMessage(content='assistant_response_0', additional_kwargs={}, example=False), HumanMessage(content='User_feedback0', additional_kwargs={}, example=False), AIMessage(content='assistant_response_1', additional_kwargs={}, example=False), HumanMessage(content='User_feedback1', additional_kwargs={}, example=False), SystemMessage(content='Shorter message.', additional_kwargs={})], message_ids=['system_message_0', 'conversation_0_assistant', 'conversation_0_

In [4]:
episode.buffer.full_memory.message_dict

{'system_message_0': [{'message': SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}),
   'rating': None},
  {'message': SystemMessage(content='Shorter message.', additional_kwargs={}),
   'rating': 'The system message is too long.'},
  {'message': SystemMessage(content='Shorter message.', additional_kwargs={}),
   'rating': 'The system message is too long.'}],
 'conversation_0_assistant': [{'message': AIMessage(content='assistant_response_0', additional_kwargs={}, example=False),
   'rating': None},
  {'message': AIMessage(content='assistant_response_0', additional_kwargs={}, example=False),
   'rating': None}],
 'conversation_0_user': [{'message': HumanMessage(content='User_feedback0', additional_kwargs={}, example=False),
   'rating': 5},
  {'message': HumanMessage(content='User_feedback0', additional_kwargs={}, example=False),
   'rating': 5}],
 'conversation_1_assistant': [{'message': AIMessage(content='assistant_response_1', additional_kwargs={}, exampl